<a href="https://colab.research.google.com/github/Marvin2798/lab-chains-in-langchain/blob/main/lab_chains_in_langchain_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain

In [2]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
!pip install python-dotenv

In [6]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [7]:
#!pip install pandas

In [9]:
import pandas as pd
df = pd.read_csv('/content/Data.csv')

In [ ]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [10]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [16]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [18]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [19]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.2)


In [20]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [21]:
# old code
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [22]:
product = "Rubber ducks"
chain.invoke(product)

AIMessage(content='Introducing the ultimate bath time companion - the Rubber Duck! This quacky little friend is perfect for adding some fun to your daily routine. With its waterproof design and adorable squeaky sound, this rubber duck will make bath time a blast for kids and adults alike. Say goodbye to boring baths and hello to endless entertainment with the Rubber Duck!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 27, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2c4358b0-6fb8-404b-89e7-a954df0ff5b7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 68, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_re

## SimpleSequentialChain

In [23]:
from langchain.chains import SimpleSequentialChain

In [24]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

<ipython-input-24-3cd7fbb7c623>:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [25]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [26]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [27]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing the latest in bath time entertainment: rubber ducks! These little quackers are perfect for adding a splash of fun to your bath routine. With their bright colors and adorable designs, they are sure to make you smile. Plus, their buoyant nature means they will never sink, no matter how hard you try to make them! So why settle for a boring bath when you can have a quacking good time with rubber ducks? Get yours today and make bath time great again!
I would rate this product a 9 out of 10. Rubber ducks are a classic bath time toy that never goes out of style, and this review highlights their fun and entertaining qualities. The review effectively conveys the joy and excitement that rubber ducks can bring to bath time, making them a great addition to any routine. The only reason I did not give it a perfect 10 is because personal preferences may vary, and some people may not find rubber ducks as enjoyable as others.

> Finished chain

{'input': 'Rubber ducks',
 'output': 'I would rate this product a 9 out of 10. Rubber ducks are a classic bath time toy that never goes out of style, and this review highlights their fun and entertaining qualities. The review effectively conveys the joy and excitement that rubber ducks can bring to bath time, making them a great addition to any routine. The only reason I did not give it a perfect 10 is because personal preferences may vary, and some people may not find rubber ducks as enjoyable as others.'}

**Repeat the above twice for different products**

## SequentialChain

In [28]:
from langchain.chains import SequentialChain

In [29]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key='fr_review' #Give a name to your output
                    )


In [30]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key='fr_short_review' #give a name to this output
                    )


In [31]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [32]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [33]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [34]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Saveur médiocre, mousse inefficace, possible contrefaçon.',
 'en_short_review': 'Mediocre flavor, ineffective foam, possible counterfeit.',
 'rating': 'I would rate the quality of the translation as a 9 out of 10. The translation is accurate and conveys the original message effectively.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [35]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [36]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [37]:
llm = ChatOpenAI(temperature=0)

In [43]:
prompt_infos = [
    {"name": "physics", "prompt_template": "Answer physics-related questions.", "description": "Helps with physics queries."},
    {"name": "history", "prompt_template": "Answer history-related questions.", "description": "Helps with history queries."}
]

destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

print("Destination Chains:")
print(destination_chains)
print("\nAvailable Destinations:")
print(destinations_str)


Destination Chains:
{'physics': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Answer physics-related questions.'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f970912bdc0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f9709115060>, root_client=<openai.OpenAI object at 0x7f970a0c1bd0>, root_async_client=<openai.AsyncOpenAI object at 0x7f9709128fa0>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), 'history': LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, temp

In [40]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [41]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [44]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain,
                         verbose=True
                        )

<ipython-input-44-d85ae20e0d46>:16: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [45]:
router_chain.invoke("Explain what principle component analysis is using linear algebra")

{'input': 'Explain what principle component analysis is using linear algebra',
 'destination': 'physics',
 'next_inputs': {'input': 'Explain the concept of principle component analysis using linear algebra'}}

In [46]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")




> Entering new MultiPromptChain chain...
physics: {'input': 'Explain principle component analysis using linear algebra'}
> Finished chain.


In [47]:
response

{'input': 'Explain principle component analysis using linear algebra',
 'text': "Sure, I'd be happy to help with any physics-related questions you have. Just let me know what you'd like to know!"}

In [ ]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."}

In [48]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
physics: {'input': 'what is the result of adding 2 and 2'}
> Finished chain.


{'input': 'what is the result of adding 2 and 2',
 'text': "Sure, I'd be happy to help with any physics-related questions you have. Just let me know what you'd like to know!"}

chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**